In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Process gene and expression data

In [ ]:
data = pd.read_excel('expression_data.xlsx', sheet_name='Data')
try:
    data = data.drop(columns=['StartV2', 'EndV2', 'Keeptot', 'Keeptrim', 'Id260210', 'OrigId'])
except:
    pass

In [ ]:
genes = (data[data['classif'].map(lambda x : x == '-')]).reset_index()
fwd = genes[genes['Strand'].map(lambda x : x == 1)]
rvs = genes[genes['Strand'].map(lambda x : x == -1)]

### Read in B. subtilis sp. 168 genome as a string

In [ ]:
genome_file = open("bsubt168.fna","r") 
genome = genome_file.read()
genome = genome.replace('\n','')[71:] #remove \n and the first part of the FASTA file

#### Define helper functions for processing the genome sequence

In [ ]:
#Get the reverse of a sequence of the form '[ATCG]*'
def reverse_complement (seq):
    conv_dic = {'A':'T', 'T':'A', 'G':'C', 'C':'G'}
    rc = ''
    for c in seq:
        rc = conv_dic[c] + rc
    return rc

#Get the sequence for a given gene
#Useful for checking if the indices in the dataframe correspond to the correct
#Genome sequence
def get_gene_sequence (gene_name, df, genome):
    gene_info = df[df['Name']==gene_name]
    start, end = (int(gene_info['StartV3']), int(gene_info['EndV3'].values))
    if gene_info['Strand'].values == -1:
        return reverse_complement(genome[start-1:end])
    else:
        return genome[start-1:end]

#Return means across different trials of the same condition and return one array of expression
#Values from all of the cells
def merge_exp_columns (df, unique_exp_columns):
    
    averaged_exps_df = pd.DataFrame()
    for exp in unique_exp_columns:
        averaged_exps_df[exp] = df.filter(regex=(exp.replace('+', '\+').replace('.','\.'))).mean(axis=1)
    return pd.DataFrame({'exps' : list(averaged_exps_df.to_numpy())}, index = df.index)

#Returns true if there are no marked genes for >buffer bp upstream of gene, else returns false
#Similar to an operon, but naming it 'unit' to emphasize that it is an artificial object
def first_in_unit (gene, df, buffer):
    strand = int(df[df['Name']==gene_name]['Strand'])
    sense_df = df[df['Strand'] == strand].reset_index()
    gene_info = sense_df[sense_df['Name']==gene_name]
    ind = gene_info.index
    if strand == 1:
        pass
    if strand == -1:
        pass

# Handle case where the sequence of interest overlaps the ends of the linear genome sequence
def get_wraparound_string (start, end, genome):
    if end > start:
        return genome[start:end]
    else:
        return genome[start:(len(genome))] + genome[0 : end]
    
# Return information for the upstream sequences for a single strand
def get_upstream_seqs_partial (single_sense_df, buffer, genome):
    ss_df = single_sense_df.sort_values(by  = "StartV3")
    ss_df = ss_df.reset_index()
    
    upstream = []
    for ind in ss_df.index:        
        curr_start = ss_df.loc[ind, "StartV3"]
        prev_end = ss_df.loc[(ind-1) % len(fwd), "EndV3"]
        if (curr_start - prev_end) % len(genome) < buffer:
            upstream.append(None)
        else:
            upst_start = (curr_start - buffer) % len(genome)
            upstream_seq = get_wraparound_string(upst_start, curr_start, genome)
            if ss_df.loc[ind, "Strand"] == 1:
                upstream.append(upstream_seq)
            elif ss_df.loc[ind, "Strand"] == -1:
                upstream.append(reverse_complement (upstream_seq))
    return pd.Series(upstream, index = single_sense_df.index)

# Return information for the upstream sequences for both DNA strands in the genome
def get_upstream_seqs_full (df, buffer, genome):
    upstream = pd.Series([])
    for strand in df['Strand'].unique():
        upstream = upstream.append(get_upstream_seqs_partial(df[df['Strand']==strand], buffer, genome))
    upstream.name = str(buffer) + "bp upstream sequence"
    return upstream

#z-score a vector of expression data for a given gene
def normalize (exp_data):
    exp_data = np.array(exp_data)
    mean = np.mean(exp_data)
    stdev = np.std(exp_data)
    return list((exp_data - mean) / stdev)

In [ ]:
exp_columns = ['LBexp_1_hyb25350202', 'LBexp_2_hyb25368302','LBexp_3_hyb25348702','LBtran_1_hyb25350002',
'LBtran_2_hyb25350302','LBtran_3_hyb25350802','LBstat_1_hyb25350502','LBstat_2_hyb25350102','LBstat_3_hyb25350702',
'LBGexp_1_hyb25351002','LBGexp_2_hyb25350402','LBGexp_3_hyb25350602','LBGtran_2_hyb27737502','LBGtran_3_hyb27061402',
'LBGstat_1_hyb27738002','LBGstat_2_hyb27737202','LBGstat_3_hyb27737302','S0_1_hyb28386102','S0_2_hyb28384902_01',
'S0_3_hyb29584102','S1_1_hyb28384002','S1_2_hyb28358602','S1_3_hyb29569102','S2_1_hyb28345202_01','S2_2_hyb28438702_01',
'S2_3_hyb29584002','S3_1_hyb28353902','S3_2_hyb28407302_01','S3_3_hyb29593702','S4_1_hyb28353602_01','S4_2_hyb29633702',
'S4_3_hyb29593902','S5_1_hyb28387702','S5_2_hyb29581902','S5_3_hyb29636602','S6_1_hyb28357602','S6_2_hyb29634602',
'S6_3_hyb29637702','S6/t_1_hyb39203702','S6/t_2_hyb42359702','S6/t_2_hyb39204702','S6_4_hyb43477102','S6_5_hyb43203402',
'S6_6_hyb43259002','S7_4_hyb43261002','S7_5_hyb43260902','S7_6_hyb43477202','S8_4_hyb43256702','S8_5_hyb43271102',
'G135_1_hyb33853802','G135_3_hyb33849902','G150_1_hyb33856402','G150_2_hyb33873202','G150_3_hyb29750102','G180_1_hyb29759602',
'G180_2_hyb29634102','G180_3_hyb33853902','C30_1_hyb42359802','C30_2_hyb42068102','C90_1_hyb39425302','C90_2_hyb39202602',
'BI_1_hyb32835602','BI_2_hyb32823102','BC_2_hyb32831902','BC/t_1_hyb39182802','BC/t_2_hyb39226502','BC_1_hyb32815402',
'Sw_1_hyb32815602','Sw_2_hyb32815502','BT_1_hyb32815202','BT_2_hyb32815702','B36_1_hyb32830302','B36_2_hyb32815102',
'B60_1_hyb32815302','B60_2_hyb32835802','M9exp_1_hyb27733402','M9exp_2_hyb27733502','M9exp_3_hyb27732902',
'M9tran_1_hyb27733002','M9tran_2_hyb27733202','M9tran_3_hyb27728702','M9stat_1_hyb27732502','M9stat_2_hyb27732802',
'M9stat_3_hyb27732702','M+G_1_hyb15402302','M+G_2_hyb15346902','M+G_3_hyb15354202','Glu_1_hyb15403702','Glu_2_hyb15349202',
'Glu_3_hyb15349002','Fru_1_hyb15354002','Fru_2_hyb15345702','Fru_3_hyb15352002','Mal_1_hyb15352102','Mal_2_hyb15350202',
'Mal_3_hyb15356102','G+S_1_hyb15362102','G+S_2_hyb15386002','G+S_3_hyb15356002','Pyr_1_hyb15521602','Pyr_2_hyb15390702',
'Pyr_3_hyb15522102','Gly_1_hyb15517802','Gly_2_hyb15356202','Gly_3_hyb15358002','Glucon_1_hyb15409902','Glucon_2_hyb15416802',
'Glucon_3_hyb15525402','aero_1_hyb32848902','aero_2_hyb32848402','aero_3_hyb32839402','nit_1_hyb32848302','nit_2_hyb32848802',
'nit_3_hyb32848702','ferm_1_hyb32835902','ferm_2_hyb32848502','ferm_3_hyb32875102','T-5.40H_1_hyb32860802','T-4.40H_1_hyb32876002',
'T-3.40H_1_hyb32860502','T-2.40H_1_hyb32875702','T-1.40H_1_hyb32874902','T-1.10H_1_hyb32875902','T-0.40H_1_hyb32860902',
'T0.0H_1_hyb32860102','T0.30H_1_hyb32876202','T1.0H_1_hyb34436402','T1.30H_1_hyb34528602','T2.0H_1_hyb34508402',
'T2.30H_1_hyb34549502','T3.0H_1_hyb34489302','T3.30H_1_hyb34489502','T4.0H_1_hyb34533902','T5.0H_1_hyb34489402',
'HPh_1_hyb11946602','HPh_2_hyb11953802','HPh_3_hyb11955202','LPh_1_hyb11962902','LPh_2_hyb11953902','LPh_3_hyb11971902',
'LPhT_1_hyb12085002','LPhT_2_hyb11955302','LPhT_3_hyb11955002','SMMPr_1_hyb25347902','SMMPr_2_hyb25348202','SMMPr_3_hyb25348302',
'HiTm_1_hyb25345002','HiTm_2_hyb25344702','HiTm_3_hyb25347002','LoTm_1_hyb25348402','LoTm_2_hyb25348102','LoTm_3_hyb25348002',
'HiOs_1_hyb25346802','HiOs_2_hyb25344902','HiOs_3_hyb25347502','BMM_1_hyb25347702','BMM_2_hyb25347602','BMM_3_hyb25344402',
'Heat_1_hyb25340602','Heat_2_hyb25344602','Heat_3_hyb25341002','Cold_1_hyb25347802','Cold_2_hyb25344502','Cold_3_hyb25340802',
'Etha_1_hyb25348602','Etha_2_hyb25340502','Etha_3_hyb25340702','SMM_1_hyb25347102','SMM_2_hyb25346902','SMM_3_hyb25346702',
'Salt_1_hyb25345102','Salt_2_hyb25344802','Salt_3_hyb25347202','Mt0_1_hyb27737602','Mt0_2_hyb27737802','Mt0_3_hyb28385002',
'M0t45_1_hyb27737402','M0t45_2_hyb28386202','M0t45_3_hyb28358702','M0t90_1_hyb27737702','M0t90_2_hyb28444802',
'M0t90_3_hyb28387802_01','M40t45_1_hyb25414802','M40t45_2_hyb28390602','M40t45_3_hyb28463502','M40t90_1_hyb27061602',
'M40t90_2_hyb28384202_01','M40t90_3_hyb28384802_01','dia0_1_hyb25321202','dia0_2_hyb25340102','dia0_3_hyb29581702',
'dia5_1_hyb25337802','dia5_2_hyb25340002','dia5_3_hyb25348502','dia15_1_hyb25321902','dia15_2_hyb25339602','dia15_3_hyb25339402',
'Diami_1_hyb27744802','Diami_2_hyb32848002','Diami_3_hyb27744702','Paraq_1_hyb29583002','Paraq_2_hyb32848102',
'Paraq_3_hyb29634702','H2O2_1_hyb32848202','H2O2_2_hyb32844902','H2O2_3_hyb27746602','Oxctl_1_hyb27746402','Oxctl_2_hyb27744902',
'Oxctl_3_hyb32839302','GM-0.1_1_hyb10236302','GM-0.2_1_hyb10240202','GM+5_1_hyb10240302','GM+15_1_hyb10242502',
'GM+25_1_hyb17179302','GM+45_1_hyb10244302','GM+60_1_hyb10244602','GM+90_1_hyb10319902','GM+120_1_hyb10322702',
'GM+150_1_hyb17158702','GM-0.1_2_hyb13973302','GM-0.2_2_hyb17239902','GM+5_2_hyb10211402','GM+10_2_hyb10216402',
'GM+15_2_hyb10226802','GM+25_2_hyb16959802','GM+45_2_hyb10229502','GM+60_2_hyb10235002','GM+90_2_hyb20971602',
'GM+150_2_hyb10236602','GM-0.1_3_hyb16991902','GM-0.2_3_hyb17183602','GM+5_3_hyb10404402','GM+10_3_hyb10410902',
'GM+15_3_hyb10216702','GM+25_3_hyb10217102','GM+45_3_hyb10218302','GM+60_3_hyb10219302','GM+90_3_hyb16981502','GM+120_3_hyb20975802',
'GM+150_3_hyb16960402','MG-0.1_1_hyb10410402','MG+5_1_hyb10226702','MG+10_1_hyb16976802','MG+15_1_hyb17176602',
'MG+25_1_hyb17138402','MG+45_1_hyb10229802','MG+120_1_hyb10237002','MG+150_1_hyb17170002','MG-0.1_2_hyb10238002','MG-0.2_2_hyb10331702',
'MG+t5_2_hyb10374502','MG+10_2_hyb10217502','MG+15_2_hyb17000602','MG+25_2_hyb10218502','MG+45_2_hyb10218602','MG+60_2_hyb10218902',
'MG+90_2_hyb10219102','MG+120_2_hyb17176202','MG-0.1_3_hyb10223302','MG-0.2_3_hyb10226602','MG+5_3_hyb20974702',
'MG+10_3_hyb10326302','MG+15_3_hyb24698102','MG+25_3_hyb14627102','MG+45_3_hyb14632602','MG+60_3_hyb17245002',
'MG+90_3_hyb14625202','MG+120_3_hyb14633902_01','MG+150_3_hyb14630502']

#Combine biological and technical replicates for the same growth condition 
unique_exps = np.unique(list(map(lambda x : x.split('_')[0], exp_columns)))

## Extract the upstream DNA sequences and process the expression data

In [ ]:
seq_lengths = [20, 50, 100, 200, 500, 1000] #upstream sequence lengths to be extracted
upst = []
for i in seq_lengths:
    upst.append(get_upstream_seqs_full (genes, i, genome))
    
exp_data = merge_exp_columns (genes, unique_exps)
norm_exp_data = exp_data['exps'].map(lambda x : normalize(x))

In [ ]:
plt.hist(np.std(np.array([i for i in exp_data.exps]), axis = 0))
plt.show()

In [ ]:
with open('experiments_order.txt', 'w') as exp_file:
    exp_file.writelines(list(map(lambda x : x + '\n', unique_exps)))

In [ ]:
all_data = pd.DataFrame({'Name': genes['Name'],
              'Expression':exp_data['exps'], 
              'Norm Expression': norm_exp_data})
for seq in upst:
    seq_len_str = str(len(seq[0]))
    all_data[seq_len_str + 'bp upstream sequence'] = seq

In [ ]:
all_data.to_csv('cleaned_data.csv')

In [ ]:
valid_gene_numb = []
for i in range (1, 1000):
    upst = get_upstream_seqs_full (genes, i, genome)
    valid_gene_numb.append(len(upst.dropna()))

In [ ]:
# visualize number of valid sequences as a function of sequence length
plt.plot(range (1, 1000), valid_gene_numb)
plt.xlabel("Upstream Sequence Length")
plt.ylabel("Number of Upstream Sequences")
plt.show()

In [ ]:
np.array(valid_gene_numb)[[19, 49, 99, 199, 499, 998]]

## Visualize Correlatedness of growth conditions

In [ ]:
indices = [str(i) for i in range(len(norm_exp_data))]


df = pd.DataFrame(data=np.array([np.array(x) for x in norm_exp_data]), index = indices, columns=unique_exps)
corrMatrix = df.corr(method='spearman')

corrMatrix.style.background_gradient()

plt.figure(figsize=(10,10))
heat_map = sns.heatmap(corrMatrix, square = True)